In [1]:
from pyppeteer import launch
import asyncio
import nest_asyncio
from bs4 import BeautifulSoup as bs
import re
import time
import pandas as pd
import requests
import urllib.request

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

nest_asyncio.apply()

In [3]:
async def get_item(categroy: str):
    # browser = await launch()
    print("Setting up broswer for web page...")
    
    browser = await launch({
            "headless": False,
        })
    page = await browser.newPage()
    
    await page.setViewport({'width': 1920, 'height': 1080})
    await page.goto(categroy, {'timeout': 10000*30})
    return page, browser

In [4]:
dir_path = "Web Crawling/data/Suning/"
p = pd.read_csv(f"{dir_path}Products_Suning.csv")
c = pd.read_csv(f"{dir_path}Customers_Suning.csv")

In [5]:
filtered = p.sort_values(['total_comment'], ascending=False).groupby(['product_type']).head(9)

In [6]:
filtered

,Unnamed: 0,product_index,product_type,product_name,prodect_brand,price,avg_rating,total_comment,seller,website
239,239,10584849457,Mobile Phone,APPLE IPHONE XR 128GB 手機 黑色,APPLE,5699.0,0,510,香港蘇寧,https://product.hksuning.com/0000000000/10584849457.html
167,167,10584849382,Mobile Phone,APPLE IPHONE XS MAX 256GB 手機 金色,APPLE,9199.0,0,509,香港蘇寧,https://product.hksuning.com/0000000000/10584849382.html
75,75,10555802003,Tablet,Apple iPad Pro 11 256GB Wi-Fi Space Gray MTXQ2ZP/A 平板電腦,Apple,6799.0,0,508,香港蘇寧,https://product.hksuning.com/0000000000/10555802003.html
302,302,10584849492,Mobile Phone,APPLE IPHONE XR 128GB 手機 白色,APPLE,5699.0,0,507,香港蘇寧,https://product.hksuning.com/0000000000/10584849492.html
285,285,10584849437,Mobile Phone,APPLE IPHONE XR 64GB 手機 黑色,APPLE,5199.0,0,507,香港蘇寧,https://product.hksuning.com/0000000000/10584849437.html
251,251,10584849318,Mobile Phone,APPLE IPHONE XS MAX 256GB 手機 太空灰,APPLE,9199.0,0,506,香港蘇寧,https://product.hksuning.com/0000000000/10584849318.html
339,339,10674060101,Mobile Phone,HUAWEI MATE 20 X 手機 藍色,HUAWEI,4988.0,0,505,香港蘇寧,https://product.hksuning.com/0000000000/10674060101.html
280,280,10584849373,Mobile Phone,APPLE IPHONE XS MAX 256GB 手機 銀色,APPLE,9199.0,0,504,香港蘇寧,https://product.hksuning.com/0000000000/10584849373.html
248,248,10584849315,Mobile Phone,APPLE IPHONE XS MAX 64GB 手機 金色,APPLE,8169.0,0,504,香港蘇寧,https://product.hksuning.com/0000000000/10584849315.html
159,159,10674060100,Mobile Phone,HUAWEI MATE 20 PRO 手機 極光色,HUAWEI,5988.0,0,504,香港蘇寧,https://product.hksuning.com/0000000000/10674060100.html


In [60]:
async def get_imgs():
    for index, row in filtered.iterrows():
        print(f"Processing product {index}")
        p_type = row['product_type'].replace(' ', '_')
        p_name = row['product_name'].replace(' ', '_')
        p_sales = str(row['total_comment'])
        p_url = row['website']

        page, browser = asyncio.get_event_loop().run_until_complete(get_item(p_url))
        source = bs(await page.content(), 'lxml').prettify()

        img_regex = re.compile(r'src="(//imgservice.suning.cn/\S+/\S+/\S+/\S+.jpg_800w_800h_4e)"/>')
        img_url = re.findall(img_regex, source)[0]
        img_path = f"{dir_path}Top-Ten-Img/{p_type}-{p_sales}.jpg"

        urllib.request.urlretrieve(f"http:{img_url}", img_path)

        print(f"Saved image in {img_path}\n")

        await browser.close()


In [59]:
asyncio.get_event_loop().run_until_complete(get_imgs())

Setting up broswer for web page...
Saved image in Web Crawling/data/Suning/Top-Ten-Img/Mobile_Phone-510.jpg

Setting up broswer for web page...
Saved image in Web Crawling/data/Suning/Top-Ten-Img/Mobile_Phone-509.jpg

Setting up broswer for web page...
Saved image in Web Crawling/data/Suning/Top-Ten-Img/Tablet-508.jpg

Setting up broswer for web page...
Saved image in Web Crawling/data/Suning/Top-Ten-Img/Mobile_Phone-507.jpg

Setting up broswer for web page...
Saved image in Web Crawling/data/Suning/Top-Ten-Img/Mobile_Phone-507.jpg

Setting up broswer for web page...
Saved image in Web Crawling/data/Suning/Top-Ten-Img/Mobile_Phone-506.jpg

Setting up broswer for web page...
Saved image in Web Crawling/data/Suning/Top-Ten-Img/Mobile_Phone-505.jpg

Setting up broswer for web page...
Saved image in Web Crawling/data/Suning/Top-Ten-Img/Mobile_Phone-504.jpg

Setting up broswer for web page...
Saved image in Web Crawling/data/Suning/Top-Ten-Img/Mobile_Phone-504.jpg

Setting up broswer for we